# Creating code coverage reports for an nbdev project

We can run tests in parallel and get coverage with [pytest-cov](https://github.com/pytest-dev/pytest-cov).

If you'd like to try this:
- you might need to use an [editable install of nbdev](https://github.com/fastai/nbdev/#installing)
- install pytest-cov and its dependencies
- copy [test_nbs.py](https://github.com/pete88b/decision_tree/blob/master/test_nbs.py) to your nbdev project
- then run `pytest --cov=[your lib name]`

Feel free to join [the discussion](https://forums.fast.ai/t/nbdev-code-coverage-n-tests/73993/6) (o:

# Overview of this module

**Note: This is probably not the best way to get coverage - but I'm leaving this content in case it's useful**

**Note:** Until the next `nbdev` release you need to use an [editable install](https://github.com/fastai/nbdev/#installing), as this module uses new functions like `split_flags_and_code`.

Feel free to use `pytest` etc but to follow these examples, you'll just need `coverage`.

This notebook creates `testcoverage.py` which is not tied to the decision_tree project (so you can just download [testcoverage.py](https://github.com/pete88b/decision_tree/blob/master/decision_tree/testcoverage.py) if you like)

Running `testcoverage.py` will:
- create a new folder in your nbdev project `[lib_path]_test`
- delete all test scripts in `[lib_path]_test`
- write a test script to `[lib_path]_test` for each notebook in `[nbs_path]`
    - and a `run_all.py` to run all test scripts in one go

To run create a test coverage report:
- cd to `nbs_path` of the project you want to test
- create test scripts with `python [full path ...]/testcoverage.py` 
- `coverage run --source=[lib_path] [lib_path]_test/run_all.py`
- `coverage report`

Creating a test coverage report for fastai2 in my env looks like:
```
cd /home/peter/github/pete88b/fastai2/nbs

python /home/peter/github/pete88b/decision_tree/decision_tree/testcoverage.py

coverage run --source=/home/peter/github/pete88b/fastai2/fastai2 /home/peter/github/pete88b/fastai2/fastai2_test/run_all.py

coverage report
```
*Note: this &uarr; fails very quickly as fastai2 tests use things that are not available in plain python.*

## What next?
- see if running tests in plain python is useful
    - it might be true that the tests of some/most projects don't need any ipython
- make artifacts (like images/mnist3.png) available to the test scripts
    - so you don't have to be in the nbs folder to run tests
- see if we can get coverage when running tests with ipython
    - this looks promising https://github.com/computationalmodelling/nbval
- see if there is a nice way to separate plain python tests and ipython tests?

## Details details ...

I chose to "import" the module being tested (rather than write all code cells to the test script) so that:
- we are testing the library created by nbdev
    - because the things we deliver are .py files, I can't help thinking that these are what we should be testing
- we could use the test scripts to test a pip installed version of the library
    - i.e. we are testing the result of the full build, package and delivery process

In [ ]:
from nbdev import *
%nbdev_default_export testcoverage

Cells will be exported to decision_tree.testcoverage,
unless a different module is specified after an export flag: `%nbdev_export special.module`


In [ ]:
%nbdev_export
from nbdev.export import *
from nbdev.imports import *

In [ ]:
%nbdev_export
def write_imports(test_file,exports):
    "write import statements to the test script for all modules exported to by the nb we're converting"
    # export is None if cell doesn't have an nbdev export flag (%nbdev_export, %nbdev_export_internal ...)
    for export in {export[0] for export in exports if export}:
        export_parts=export.split('.')
        b=export_parts.pop()
        export_parts.insert(0,Config().lib_name)
        a='.'.join(export_parts)
        test_file.write(f"""
from {a} import {b}
for o in dir({b}):
    exec(f'from {a}.{b} import {{o}}')
        """)

the test scipt will import everything returned by `dir(module)` because we need the test code to run as if it's in the module we're testing

In [ ]:
%nbdev_export
def write_test_cell_callback(i,cell,export,code):
    "Return the code to be written to the test script or `None` to not write anything for `cell`"
    things_to_exclude = ['notebook2script','show_doc']
    if export: return None # if it's exported to the library, don't add to test script
    for thing_to_exclude in things_to_exclude: # TODO: make this better
        if thing_to_exclude in code: return None 
    return re.sub(r'^\s*(%|!)', r'#\1', code, flags=re.MULTILINE)

In [ ]:
%nbdev_export
def write_test_cells(test_file,nb,exports):
    "Writes the source of code cells to the test script"
    sep = '\n'* (int(Config().get('cell_spacing', '1'))+1)
    cells = [(i,c,e) for i,(c,e) in enumerate(zip(nb['cells'],exports)) if c['cell_type']=='code']
    for i,c,e in cells:
        code_lines = split_flags_and_code(c)[1]
        code = sep + '\n'.join(code_lines)
        code = re.sub(r' +$', '', code, flags=re.MULTILINE)
        code = write_test_cell_callback(i,c,e,code)
        if code: test_file.write(code)

In [ ]:
%nbdev_export
def notebook2testscript():
    "Convert notebooks to test scripts"
    test_path=Path(str(Config().lib_path)+'_test')
    test_path.mkdir(parents=True, exist_ok=True)
    for old_file in test_path.glob('test_*.py'): old_file.unlink()
    print('Removed all test_*.py files from',test_path)
    files = [f for f in Config().nbs_path.glob('*.ipynb') if not f.name.startswith('_')]
    for nb_file in sorted(files): 
        test_file_name = test_path/f'test_{nb_file.stem.replace("-","_")}.py'
        print('Converting', nb_file.name, 'to\n  ', test_file_name)
        file_path = os.path.relpath(nb_file, Config().config_file.parent).replace('\\', '/')
        with open(test_file_name, 'w', encoding='utf8') as test_file:
            test_file.write(f"# AUTOGENERATED! DO NOT EDIT! File to edit: {file_path} (unless otherwise specified).\n")
            nb=read_nb(nb_file)
            default_export=find_default_export(nb['cells'])
            exports = [is_export(c, default_export) for c in nb['cells']]
            write_imports(test_file,exports)
            write_test_cells(test_file,nb,exports)
    print('Writing',test_path/'run_all.py')
    with open(test_path/'run_all.py', 'w', encoding='utf8') as run_all_file:
        for nb_file in sorted(files): run_all_file.write(f'import test_{nb_file.stem.replace("-","_")}\n')

In [ ]:
%nbdev_export
if __name__ == "__main__" and not IN_NOTEBOOK:
    notebook2testscript()

In [ ]:
%nbdev_hide
notebook2script()

Converted 000_target_module.ipynb.
Converted 001_exports_to_target_module.ipynb.
Converted 002_target_module.ipynb.
Converted 00_core.ipynb.
Converted 10_data.ipynb.
Converted 20_models.ipynb.
Converted 21_models-extra.ipynb.
Converted 30_test_flag.ipynb.
Converted 40_test_export.ipynb.
Converted 50_test_doc.ipynb.
Converted 51_test_show_doc.ipynb.
Converted 60_all_test.ipynb.
Converted 61_test_add2__all__.ipynb.
Converted 70_multi_all_test_flag.ipynb.
Converted 71_tensor_patch.ipynb.
Converted 72_if__name__.ipynb.
Converted 73_in_ipython.ipynb.
Converted 80_test_coverage.ipynb.
Converted 81_test_coverage.ipynb.
Converted index.ipynb.


# PB notes
- convert all notebooks that don't start with `_` 
    - import default_export of notebook
    - import things exported to other modules 
    - handle nbdev test flags - TODO
    - create `test_[notebook name].py`
    - write code of test cells to `test_[notebook name].py`
        - exclude show_doc, notebook2script, cmd calls etc TODO

```
coverage run --source=/home/peter/github/pete88b/decision_tree/decision_tree /home/peter/github/pete88b/decision_tree/decision_tree_test/test_00_core.py

coverage run --source=/home/peter/github/pete88b/decision_tree/decision_tree /home/peter/github/pete88b/decision_tree/decision_tree_test/run_all.py
```

```
cd /home/peter/github/pete88b/decision_tree

python /home/peter/github/pete88b/decision_tree/decision_tree/testcoverage.py

coverage run --source=/home/peter/github/pete88b/decision_tree/decision_tree /home/peter/github/pete88b/decision_tree/decision_tree_test/run_all.py

coverage report
```